In [1]:
import time

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

# Spark Streaming
Spark supports near real-time streams of data using what it calls a Discretized (DStream). DStreams are an abstraction layer that represents a continous stream of data. This means that we need some kind of event queuing system such as Kafka serving events in order to fully realize Spark's real-time capabilities. In this lab, we'll simulate an event queue by creating a directory, and having Spark listen for when this directory gets updated.

## Setup
Before we dive too deeply into Spark Streaming, we first need to instantiate a SparkSession() and configure it to not output parquet file metadata. Recall back to when we learned how to serialize files in 2_1. If you looked at the directory that gets materialized, you would have noticed that Spark writes several metadata files. We can disable this by setting `parquet.enable.summary-metadata` to "false". Next, you probably also noticed a file that takes up 0 bytes simply called SUCCESS. This file is basically a marker that lets us know the files were written successfully. This is useful when launching jobs on a cluster, but for simple local runs, it's safe to disable. To do that, we'll also set `mapreduce.fileoutputcommitter.marksuccessfuljobs` to "false".

In [2]:
spark = SparkSession(sc)
spark.conf.set("parquet.enable.summary-metadata", "false")
spark.conf.set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

As finally bit of setup, we'll go ahead and use our newly created SparkSession to create a DataFrame from our long file. In this case, Spark is smart enough to split the data newline, so we don't need to specify a schema.

In [3]:
df = spark.read.text('../data/long_file.txt')

## Introducing the StreamingContext
Just like batch processing, streaming processing requires a context to interact with the cluster manager and worker nodes. Similar to a SparkSession, a StreamingContext relies on a SparkContext. Also similar to a SparkContext, you may only have one active StreamingContext per Spark job. In this case, we'll create a StreamingContext, and set its batch polling interval to 5 seconds. Setting the batck polling interval basically establishes how often the StreamingContext will check the event queue for new files

In [5]:
ssc = StreamingContext(sc, batchDuration=5)

## Instantiating a DStream
DStreams support several different kinds of streaming sources. Here's a nonexhaustive list:

* socketTextStream - Listens for data streaming from a TCP source
* textFileStream - Listens for files being added to a target directory
* queueStream - Listens for RDDs being added to a queue of RDD's

For this lab, we'll implement a textFileStream that listens for new files being created in a target directory called `event_queue`

In [ ]:
words_stream = ssc.textFileStream('../data/event_queue/')

Next, we'll implement a basic word count, and then print our results using the built-in method `pprint()`. pprint() is a method provided by the StreamingContext that allows us to print out the results of any data that was processed during a given polling interval.

In [ ]:
words_rdd = words_stream.flatMap(lambda x: x.split()).filter(str.isalpha).filter(lambda x: len(x) > 3)
reduced_rdd = words_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
reduced_rdd.pprint()

## Streaming data
We intialize a stream by calling `start()` on our StreamingContext. The moment this method executes, the textFileStream will begin watching the event_queue directory for new files until we call the `stop()` method. Note that calling stop() normally will also unload the SparkContext.

Since Jupyter treats each cell as a blocking operation by default, this portion of the lab is implemented all in the same cell. We first call `start()` to start the stream. Next, we randomly sample .1% of the DataFrame we created earlier, and then we write the sample in parquet format to the event_queue every five seconds. Finally, we close the stream, leaving the SparkContext active

In [6]:
ssc.start()

for i in range(5):
    sample = df.sample(0.001)
    sample.write.format('parquet').mode('overwrite').save('../data/event_queue/')
    time.sleep(5)

ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2021-02-04 23:25:25
-------------------------------------------

-------------------------------------------
Time: 2021-02-04 23:25:35
-------------------------------------------
('LUCIUS', 2)
('with', 5)
('army', 2)
('GOTHS', 2)
('drums', 2)
('Whose', 1)
('fresh', 1)
('repair', 1)
('thou', 1)
('labouring', 1)
...

-------------------------------------------
Time: 2021-02-04 23:25:40
-------------------------------------------

-------------------------------------------
Time: 2021-02-04 23:25:45
-------------------------------------------
('COPYRIGHT', 3)
('WORLD', 3)
('what', 1)
('doth', 1)
('thee', 1)
('gave', 1)
('breaking', 1)
('Spanish', 1)
('Thracian', 1)
('bids', 1)
...

-------------------------------------------
Time: 2021-02-04 23:25:50
-------------------------------------------
('SCENE', 2)
('seemly', 1)
('raiment', 1)
('those', 1)
('nine', 1)
('which', 1)
('rhymers', 1)
('Into', 1)
('rvice', 1)
('What', 1)
...

-----------